In [18]:
from bs4 import BeautifulSoup
import pandas as pd
import psycopg2
from lxml.html import parse
import csv
import io
from sqlalchemy import create_engine

In [40]:
from lxml.html import parse
page = parse("http://www.brooksbaseball.net/pfxVB/tabdel_expanded.php?pitchSel=403178&game=gid_2017_08_01_kcamlb_balmlb_1/&s_type=3&h_size=700&v_size=500")
rows = page.xpath("body/table")[0].findall("tr")
data = list()
for row in rows:
    data.append([c.text for c in row.getchildren()])
    print(data)
    if len(data) == 1:
        print("FUCK!")
    
pitcher_df = pd.DataFrame(data, columns = ["dateStamp", "park_sv_id", "play_guid", "ab_total", "ab_count", "pitcher_id", "batter_id", "ab_id", "des", "pitch_type", "pitch_id", "sz_top", "sz_bot", "pfx_xDataFile", "pfx_zDataFile", "mlbam_pitch_name", "zone_location", "pitch_con", "stand", "strikes", "balls", "p_throws", "gid", "pdes", "spin", "norm_ht", "inning", "pitcher_team", "tstart", "vystart", "ftime", "pfx_x", "pfx_z", "uncorrected_pfx_x", "uncorrected_pfx_z", "x0", "y0", "z0", "vx0", "vy0", "vz0", "ax", "ay", "az", "start_speed", "px", "pz", "pxold", "pzold", "tm_spin", "sb"])
pitcher_df

[['result not valid']]
FUCK!


AssertionError: 51 columns passed, passed data had 1 columns

In [19]:
def write_to_table(df, db_engine, table_name, if_exists='fail'):
    string_data_io = io.StringIO()
    df.to_csv(string_data_io, sep='|', index=False)
    pd_sql_engine = pd.io.sql.pandasSQL_builder(db_engine)
    table = pd.io.sql.SQLTable(table_name, pd_sql_engine, frame=df,
                               index=False, if_exists=if_exists)
    table.create()
    string_data_io.seek(0)
    string_data_io.readline()  # remove header
    with db_engine.connect() as connection:
        with connection.connection.cursor() as cursor:
            copy_cmd = "COPY %s FROM STDIN HEADER DELIMITER '|' CSV" % table_name
            cursor.copy_expert(copy_cmd, string_data_io)
        connection.connection.commit()

In [20]:
address = 'postgresql://@localhost:5432/pfxbaseballdata'
engine = create_engine(address)

In [21]:
write_to_table(pitcher_df, engine, 'test_batters', "replace")

In [12]:
pitcher_df[1:].to_csv('pitcher_table.csv', encoding='utf-8',index=False)

In [13]:
! head pitcher_table.csv

dateStamp,park_sv_id,play_guid,ab_total,ab_count,pitcher_id,batter_id,ab_id,des,pitch_type,pitch_id,sz_top,sz_bot,pfx_xDataFile,pfx_zDataFile,mlbam_pitch_name,zone_location,pitch_con,stand,strikes,balls,p_throws,gid,pdes,spin,norm_ht,inning,pitcher_team,tstart,vystart,ftime,pfx_x,pfx_z,uncorrected_pfx_x,uncorrected_pfx_z,x0,y0,z0,vx0,vy0,vz0,ax,ay,az,start_speed,px,pz,pxold,pzold,tm_spin,sb
2017-04-07,170407_002229hou,0a931d62-12e2-4fb9-b2ff-09dfdd551ecf,6,1,664641,543807,5,Home Run,B,38,3.9611,1.93801,6.1876,10.5233,FF,19,0.856,R,0,0,L,gid_2017_04_06_seamlb_houmlb_1/,Ball,149.575,-1.08779,1,sea,0.07666821236504766,-129.35233735237819,0.3086538920745674,6.7008325784929355,11.000371853418763,6.1876,10.5233,1.7550111362006353,55,6.532892350051115,-2.9783688495463134,-132.5785925448279,-7.981929300344492,11.72287141418457,28.168534088134766,-12.934235527038574,90.58,1.5443087965382278,1.9968904054487366,1.59942,1.84921,0,00
2017-04-07,170407_002225_06,0a931d62-12e2-4fb9-b2ff-09dfdd551ecf,